In [1]:
import os
import openai
import configparser

conf = configparser.ConfigParser()
conf.read("../config.ini")
openai.api_key = conf.get("Openai", "api_key")
os.environ["HTTP_PROXY"] = conf.get("Proxy", "HTTP_PROXY")  # 配置自己的代理
os.environ["HTTPS_PROXY"] = conf.get("Proxy", "HTTPS_PROXY")


In [2]:
all_models = openai.Model.list()
all_models = [model.get('id') for model in all_models.get("data")]
print(all_models, len(all_models))

['text-davinci-001', 'text-search-curie-query-001', 'davinci', 'text-babbage-001', 'curie-instruct-beta', 'text-davinci-003', 'gpt-3.5-turbo-16k-0613', 'davinci-similarity', 'code-davinci-edit-001', 'text-similarity-curie-001', 'text-embedding-ada-002', 'gpt-3.5-turbo-16k', 'ada-code-search-text', 'text-search-ada-query-001', 'babbage-search-query', 'ada-similarity', 'gpt-3.5-turbo', 'whisper-1', 'text-search-ada-doc-001', 'text-search-babbage-query-001', 'code-search-ada-code-001', 'curie-search-document', 'text-search-davinci-query-001', 'text-search-curie-doc-001', 'gpt-3.5-turbo-0301', 'babbage-search-document', 'babbage-code-search-text', 'davinci-instruct-beta', 'davinci-search-query', 'text-similarity-babbage-001', 'text-davinci-002', 'code-search-babbage-text-001', 'babbage', 'text-search-davinci-doc-001', 'code-search-ada-text-001', 'ada-search-query', 'text-similarity-ada-001', 'ada-code-search-code', 'ada', 'text-davinci-edit-001', 'davinci-search-document', 'curie-search-qu

In [10]:
text_model = 'text-davinci-003'
chat_model = "gpt-3.5-turbo"

In [3]:
openai.Model.retrieve(chat_model)  # 获取单个模型详细信息

<Model model id=gpt-3.5-turbo at 0x107a48b30> JSON: {
  "id": "gpt-3.5-turbo",
  "object": "model",
  "created": 1677610602,
  "owned_by": "openai",
  "permission": [
    {
      "id": "modelperm-zy5TOjnE2zVaicIcKO9bQDgX",
      "object": "model_permission",
      "created": 1690864883,
      "allow_create_engine": false,
      "allow_sampling": true,
      "allow_logprobs": true,
      "allow_search_indices": false,
      "allow_view": true,
      "allow_fine_tuning": false,
      "organization": "*",
      "group": null,
      "is_blocking": false
    }
  ],
  "root": "gpt-3.5-turbo",
  "parent": null
}

In [4]:
openai.Model.retrieve(text_model)  # 获取单个模型详细信息

<Model model id=text-davinci-003 at 0x10a24a390> JSON: {
  "id": "text-davinci-003",
  "object": "model",
  "created": 1669599635,
  "owned_by": "openai-internal",
  "permission": [
    {
      "id": "modelperm-a6niqBmW2JaGmo0fDO7FEt1n",
      "object": "model_permission",
      "created": 1690930172,
      "allow_create_engine": false,
      "allow_sampling": true,
      "allow_logprobs": true,
      "allow_search_indices": false,
      "allow_view": true,
      "allow_fine_tuning": false,
      "organization": "*",
      "group": null,
      "is_blocking": false
    }
  ],
  "root": "text-davinci-003",
  "parent": null
}

## OpenAI上下文补全Completions与ChatCompletions API

In [6]:
# Completion
conversation = openai.Completion.create(
    model=text_model,
    prompt='You are a translation assistant',
    max_tokens=10,
    temperature=0.2,
    stream=False
)
text = conversation["choices"][0]['text']
print(text)



As a translation assistant, you would be


In [8]:
# Completion
conversation = openai.Completion.create(
    model=chat_model,
    prompt='You are a translation assistant',
    max_tokens=10,
    temperature=0.2,
    stream=False
)
text = conversation["choices"][0]['text']
print(text)

InvalidRequestError: This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?

In [17]:
conversation_2 = openai.Completion.create(
    model=text_model,
    prompt=text,
    max_tokens=10,
    temperature=0.2,
    stream=False
)

In [19]:
text2 = conversation_2["choices"][0]['text']
print(text2)

 to help the translator with the translation process. You


In [36]:
# ChatCompletion
message = [
    {"role": "system", "content": "You are a translation assistant, and you name is Macy, a kindness young lady."},
    {"role": "user", "content": "Hi Macy! I am glad to see you."}
]
conversation_3 = openai.ChatCompletion.create(
    model=chat_model,
    messages=message,
    max_tokens=50,
    temperature=0.2,
    stream=True
)
for event in conversation_3:
    if event['choices'][0]['finish_reason'] != 'stop':
        print(event['choices'][0]['delta']['content'], end='')

Hello! I'm delighted to see you too. How can I assist you today?

In [12]:
# Completion
message = [
    {"role": "system", "content": "You are a translation assistant, and you name is Macy, a kindness young lady."},
    {"role": "user", "content": "Hi Macy! I am glad to see you."}
]
conversation = openai.ChatCompletion.create(
    model=chat_model,
    messages=message,
    max_tokens=100,
    temperature=0.2,
    # stream=False
)
for event in conversation:
    print(event['choices'])
    if event['choices'][0]['finish_reason'] != 'stop':
        print(event['choices'][0]['delta']['content'], end='')

TypeError: string indices must be integers